| Name | Type (Shape) | Description |
| --- | --- | --- |
| time | array (N,) | time |
| x | array (3,) | $ \mathbf{x} $ |
| X | array (NY, NX, NZ) | X-meshgrid|
| Y | array (NY, NX, NZ) | Y-meshgrid|
| Z | array (NY, NX, NZ) | Z-meshgrid|
| Interpolant_u | RectangularGridInterpolator | Interpolant object for $ u(\mathbf{x}, t)  $ |
| Interpolant_v | RectangularGridInterpolator | Interpolant object for $ v(\mathbf{x}, t)  $ |
| Interpolant_w | RectangularGridInterpolator | Interpolant object for $ w(\mathbf{x}, t)  $ |
| periodic | list (3,) | periodic[0]: periodicity in x <br /> periodic[1]: periodicity in y <br /> periodic[2]: periodicity in z |
| bool_unsteady | bool | specifies if velocity field is unsteady/steady |
| Fmap | array (3,N) | integrated trajectory (=flow map) |
| dFdt | array (3,N) | velocity along trajectories (=time derivative of flow map) |

In [1]:
# import sys/os
import sys, os

# get current directory
path = os.getcwd()

# get parent directory
parent_directory = os.path.sep.join(path.split(os.path.sep)[:-1])

# add utils folder to current working path in order to access the functions
sys.path.append(parent_directory+"/utils")

In [3]:
# Import numpy
import numpy as np

# Import function to compute velocity
from ipynb.fs.defs.velocity import velocity

In [4]:
def integration_dFdt(time, x, X, Y, Z, Interpolant_u, Interpolant_v, Interpolant_w, periodic, bool_unsteady, verbose = False):
    '''
    Wrapper for RK4_step(). Advances the flow field given by u, v, w velocities, starting from given initial conditions. 
    The initial conditions can be specified as an array. 
    
    Parameters:
        time: array (Nt,),  time array  
        x: array (3, Npoints),  array of ICs
        X: array (NY, NX, NZ)  X-meshgrid
        Y: array (NY, NX, NZ)  Y-meshgrid 
        Z: array (NY, NX, NZ)  Z-meshgrid
        Interpolant_u: Interpolant object for u(x, t)
        Interpolant_v: Interpolant object for v(x, t)
        Interpolant_w: Interpolant object for w(x, t)
        periodic: list of 3 bools, periodic[i] is True if the flow is periodic in the ith coordinate.
        bool_unsteady:  specifies if velocity field is unsteady/steady
        verbose: bool, if True, function reports progress
    
    Returns:
        Fmap: array (Nt, 3, Npoints), integrated trajectory (=flow map)
        dFdt: array (Nt-1, 3, Npoints), velocity along trajectories (=time derivative of flow map) 
    '''
    # reshape x
    x = x.reshape(3, -1) # reshape array (3, Nx*Ny*Nz)
    
    # Initialize arrays for flow map and derivative of flow map
    Fmap = np.zeros((len(time), 3, x.shape[1])) # array (Nt, 3, Nx*Ny*Nz)
    dFdt = np.zeros((len(time)-1, 3, x.shape[1])) # array (Nt-1, 3, Nx*Ny*Nz)
    
    # Step-size
    dt = time[1]-time[0] # float
    
    counter = 0 # int

    # initial conditions
    Fmap[counter,:,:] = x
    
    # Runge Kutta 4th order integration with fixed step size dt
    for t in time[:-1]:
        if verbose == True:
            if np.around((t-time[0])/(time[-1]-time[0])*1000,4)%10 == 0:
                print('Percentage completed: ', np.around(np.around((t-time[0])/(time[-1]-time[0]), 4)*100, 2))
        
        Fmap[counter+1,:, :], dFdt[counter,:,:] = RK4_step(t, Fmap[counter,:, :], dt, X, Y, Z, Interpolant_u, Interpolant_v, Interpolant_w, periodic, bool_unsteady)[:2]
    
        counter += 1
    
    return Fmap, dFdt

In [ ]:
def RK4_step(t, x1, dt, X, Y, Z, Interpolant_u, Interpolant_v, Interpolant_w, periodic, bool_unsteady):
    '''
    Advances the flow field by a single step given by u, v, w velocities, starting from given initial conditions. 
    The initial conditions can be specified as an array. 
    
    Parameters:
        time: array (Nt,),  time array  
        x: array (3,Npoints),  array of ICs
        X: array (NY, NX, NZ)  X-meshgrid
        Y: array (NY, NX, NZ)  Y-meshgrid 
        Z: array (NY, NX, NZ)  Z-meshgrid
        Interpolant_u: Interpolant object for u(x, t)
        Interpolant_v: Interpolant object for v(x, t)
        Interpolant_w: Interpolant object for w(x, t)
        periodic: list of 3 bools, periodic[i] is True if the flow is periodic in the ith coordinate.
        bool_unsteady:  specifies if velocity field is unsteady/steady
    
    Returns:

        y_update: array (3,Npoints), integrated trajectory (=flow map) 
        y_prime_update: array (3,Npoints), velocity along trajectories (=time derivative of flow map) 
    '''
    t0 = t # float
    
    # Compute x_prime at the beginning of the time-step by re-orienting and rescaling the vector field
    x_prime = velocity(t, x1, X, Y, Z, Interpolant_u, Interpolant_v, Interpolant_w, periodic, bool_unsteady) # array(3, Nx*Ny*Nz)
    
    # compute derivative
    k1 = dt * x_prime # array(3, Nx*Ny*Nz)
    
    # Update position at the first midpoint.
    x2 = x1 + .5 * k1 # array(3, Nx*Ny*Nz)
     
    # Update time
    t = t0 + .5*dt # float
    
    # Compute x_prime at the first midpoint.
    x_prime = velocity(t, x2, X, Y, Z, Interpolant_u, Interpolant_v, Interpolant_w, periodic, bool_unsteady) # array(3, Nx*Ny*Nz)
    
    # compute derivative
    k2 = dt * x_prime # array(3, Nx*Ny*Nz)

    # Update position at the second midpoint.
    x3 = x1 + .5 * k2 # array(3, Nx*Ny*Nz)
    
    # Update time
    t = t0 + .5*dt # float
    
    # Compute x_prime at the second midpoint.
    x_prime = velocity(t, x3, X, Y, Z, Interpolant_u, Interpolant_v, Interpolant_w, periodic, bool_unsteady) # array(3, Nx*Ny*Nz)
    
    # compute derivative
    k3 = dt * x_prime # array(3, Nx*Ny*Nz)
    
    # Update position at the endpoint.
    x4 = x1 + k3 # array(3, Nx*Ny*Nz)
    
    # Update time
    t = t0+dt # float
    
    # Compute derivative at the end of the time-step.
    x_prime = velocity(t, x4, X, Y, Z, Interpolant_u, Interpolant_v, Interpolant_w, periodic, bool_unsteady) # array(3, Nx*Ny*Nz)
    
    # compute derivative
    k4 = dt * x_prime
    
    # Compute RK4 derivative
    y_prime_update = 1.0 / 6.0*(k1 + 2 * k2 + 2 * k3 + k4) # array(3, Nx*Ny*Nz)
    
    # Integration y <-- y + y_primeupdate
    y_update = x1 + y_prime_update # array(3, Nx*Ny*Nz)
    
    return y_update, y_prime_update/dt